In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing import image

In [ ]:
def ConvBlock(model, layers, filters):
    for i in range(layers):
        model.add(Conv2D(filters, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model = Sequential()
model.add(BatchNormalization(input_shape=(480,640,3), axis=1))
ConvBlock(model, 2, 64)
ConvBlock(model, 2, 128)
ConvBlock(model, 2, 256)
ConvBlock(model, 3, 512)
ConvBlock(model, 3, 1024)
          
model.add(Flatten())
model.add(Dense(4000, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:

train_batches_augmented = image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True
).flow_from_directory(
    '/data/cervical/train', 
    target_size=(480, 640),
    class_mode='categorical', 
    shuffle=True, 
    batch_size=16
)

sample_valid_batches = image.ImageDataGenerator().flow_from_directory(
    '/data/cervical/valid', 
    target_size=(480, 640),
    class_mode='categorical', 
    shuffle=True, 
    batch_size=8
)

In [ ]:
model.fit_generator(train_batches_augmented, 100,
                 epochs=3, 
                 validation_data=sample_valid_batches,
                 validation_steps=19
                )